In [2]:
import pandas as pd
import sqlalchemy
import psycopg2


In [3]:
db_config = {
    'user': 'postgres',
    'password': '12345678',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres'
}

In [4]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

In [5]:
df = pd.read_csv("brands.csv")

In [6]:
df

,barcode,category,categoryCode,name,topBrand,brandCode,brandId,cpgId,cpgRef
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,False,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,LIPTON TEA Leaves,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs


In [7]:
timestamp_columns = ['createdDate', 'lastLogin']
for col in timestamp_columns:
    if col in df.columns:
        df[col] = df[col].where(pd.notnull(df[col]), None)

df['topBrand'] = df['topBrand'].apply(lambda x: None if pd.isna(x) else x)

table_name = 'brands'

for index, row in df.iterrows():
    cursor.execute(
        f"INSERT INTO Fetchrewards.{table_name} (barcode, category, categoryCode, name, topBrand, brandCode, brandId, cpgId, cpgRef) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (row['barcode'], row['category'], row['categoryCode'], row['name'], row['topBrand'], row['brandCode'], row['brandId'],row['cpgId'],row['cpgRef'])
    )
conn.commit()